In [39]:
from api.riot_api_routes import *
from api.variables import SERVER_ROUTES, DIVISIONS, TIERS, GAME_TYPE,MAIN_GAMEMODE,RANKED_QUEUES

import json
from django.conf import settings
from os import path
import os
import random

import pandas as pd
from time import time, sleep

league_data_route=path.join(settings.PREPROCESS_PATH, "league_data.json")
player_sample_route=path.join(settings.PREPROCESS_PATH, "player_sample.json")
matches_sample_route=path.join(settings.PREPROCESS_PATH, "matches_sample.csv")


## Cantidad de jugadores

In [2]:
print(player_sample_route)

/home/facundo/Desktop/Code/projects/lol_app_2/lol_stats_api/preprocess/player_sample.json


In [3]:
if not path.isfile(player_sample_route):
    print("No se encuentra el archivo")

with open(player_sample_route, "r") as f:
    player_data = json.load(f)

In [4]:
len(player_data)

1480

## Por cada jugador busco sus ultimas 10 partidas

In [5]:
df = pd.DataFrame(player_data)
df

,summonerId,tier,leaguePoints,wins,losses,summonerName,server,division,winrate,accountId,puuid,profileIconId,summonerLevel,revisionDate
0,zT--AZycIeIL-ThYeLbsU7A2DaqpxRuu9lY0jEVYGIpzZA,I,502,110,74,H4O,LAS,challengers,59.8,x76TyZjA_DwYjz15R8l2EJE4EunjCSjNhuRcwwr5adVTmXA,kxnvTXExlu-6hsowbTn7kHW-MCDtyG34ROE1tbLhAtjdO9...,3799,122,1586128468000
1,tcjCEKXC0Q17Q4hIIgUWp9K7aNSmvH79L8JhU9iRBfn_oQ,I,438,187,176,S4MI,LAS,challengers,51.5,uD-7mhoKz_8cbxPETK9TF_FTMlni0YjTv9oXliazwHiNuNo,F1Fmo4HlE85oQW5xTmamZcD4WAsvGbrRB1fjsg0UeqgSh3...,4512,283,1586610513000
2,DoGGqqLGGJkp-3_0e-4Yw-fsd9W691Z58UYjJcNgGDtzhg,I,632,208,170,męow,LAS,challengers,55.0,sWLnG-EQQGG6pFfuoSID3oZ12B0_MvS5nAW-yWfla39dvog,oDsaf91SKn8SPZMfpDvWxM_g2vru4DvWZS1oEgCO5aGT7k...,4533,340,1586531658000
3,MFh2bm52RrgLJdXAiSFj8k8i2w--ovm6H1RszfWGsm5Taz8,I,721,92,65,BibleThump,LAS,challengers,58.6,sfxKGbuclnsoenBZErGEbYd2Ll6NOphfjdxzRCel1jzizGw,RoCAYeRDbLLHt3ouZwoXkZseBBz-Dugw2RE75kzuQkzLcH...,6,119,1586355749000
4,lz9fDbh3QPFeelZQZZfX--27nEsNbp2S0W0xxyzt4dwh,I,881,111,82,GALAXY IQ BRAIN,LAS,challengers,57.5,W3PeEhT-upcDUCkbsY0Ha96dwU8VbS2ayqOdI6WcEd8,L3EYm7OU5i1RJ_MwIiHje2UfOsOYnfi7Z0Wk3xbZ1kro4n...,3797,223,1586388376000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1475,PDp9b8nyJDqB6HJtTzY-f-mFOBaLl4IRUKd9h3AwiGSxK_k,IV,70,12,16,Melon,RU,IRON,42.9,a5GNuT8AG2pEzfTjqteXYkng00RcLHgq0FwAYcMrz1tMaKs,aIZaQ7cz8f8xWIa72FCpST35-iznqxZavvjzxhY2vyffh1...,4448,93,1579445112000
1476,e6xGVn4u6axB6hWYHvWhS3AmSXzuM8A3i0PbiYSfa1FgTRQ,IV,40,2,16,3Джайна3,RU,IRON,11.1,HUZBdLP0UgRdl9d4FKxQBUQ-UBiupjVVZJCY8qlmStZalsY,Pg6eDd875Ln9DriCnZ_fyK6lH6bgLsUeI_BxTjcuXwWi6W...,4535,156,1586495910000
1477,9KLVO4BJjfH84Zp0RMIWye7pzXuMiQ-Htd0EnSMOR4TX0Q,IV,67,2,11,Endlessness,RU,IRON,15.4,MEi0vzKNVKbhCdXYr9KFj-_fSHP8NqYZXRGitCQMQOqEt_s,73_-SpIk6P5uedCxP6Auh4YehqPbq0QW0qVXyWfPMFbPl3...,1429,61,1586371677000
1478,nRbQql5vgzyDT02_LDuYdmn0c-jDtpfq_1JHlDK_Y__4NA,IV,0,21,31,primera2,RU,IRON,40.4,k3TtIEDfnVi-VFcRbS1WZ8oVUg7-PO9ae0ExM0tHg7yk7To,NKSHecuvJtAmCDGLfFTtAXW63dK9O_aZglAsqGkfCWtmUZ...,4536,61,1586623078000


In [6]:
#Desordeno el dataframe para evitar agotar el maximo de requests
df = df.sample(frac=1).reset_index(drop=True)
df

,summonerId,tier,leaguePoints,wins,losses,summonerName,server,division,winrate,accountId,puuid,profileIconId,summonerLevel,revisionDate
0,fczOqmrbFCjbB0RgmfBNlDOrTP8uCw1uyoENhRcvacIP7Ac,II,100,47,39,Sobolewski,BR,DIAMOND,54.7,j8Bs-fgM6NiMezttZrrWTbzORuJFYnaFhpb6SesUDzFwzbk,rxRLY_PK0SMD6FX5NAaG3CcuwcXnXqhIviqPpvOhd9AzWo...,4491,184,1586060771000
1,gygdnDduuKueOqmlfRw7lVOH3HIwKQkJrkDxOsbom27Mfc0,IV,16,17,5,소환사5103,KR,SILVER,77.3,p0-eRHOI-H-aGp0b9aTPf1rOZQ4ryN-YYbp-Bpacv151,ZiZzzxJ83CbaQZJZheUYCCrTJHKKxhaadCh-WIztV2CvZN...,3551,72,1586451515000
2,KpKH5SWvJHtqHQEmSetyuTmiomMqiR08olOvdVb5cX5n,I,651,230,200,DRAGKUDRAGKUDRAG,OC,challengers,53.5,JDZ8T87YaOXydeNvHiYQ459vTgZPc0Z2lZJH0L2V6k0SXjA,T9pV_kzGoV4Ea_LswH5PJ-9RhJh998v7YbOmcwTMdD2eYe...,4534,331,1586062310000
3,Qw4lE9jvvCeDgFpqPISj-FGXwDQgsavu0btUg7L58dsA,III,16,17,23,Sailsos,EUN,BRONZE,42.5,2XWf2Q0UgZq-J73xfKDccHdP3OC8OI0QzCSglQADZ-k,txD5hT078i_zRq2FKqcXYXF35CNTIFj4B7IzX2iDT_4_6l...,898,58,1586419560000
4,jBYOf0YazILpAXx4Yb_Oi3nNasmBdgGdqPh-LBbOzqMOz-g,III,76,5,9,DeltaPlayer,TR,IRON,35.7,MxV3utPKctfDYrilr6igg2V2976NoElUIYg4An_DAKi4AMo,m2C8l0UjEguQ9i6v688WmF2tgPJTvUkwYVsjdHvS_ZperU...,3006,76,1586360269000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1475,udtOgiXQ-HvvojPkGtocWcH8Df6qzDZxeGYVPfb1XzzYaw,I,51,56,61,AngelDacy,LAS,DIAMOND,47.9,HifuDqespF3gNs6mGof7AcGrxz_Jgf3Cp5EAFFPDzUO4,20XebZhI6gs5lcrFBGDXcb8Ht9S01ctwH-2V088d_3gmRp...,1625,469,1586115761000
1476,W7AhJNksi3X4IzVpBW4grcbRb6ZRsQM9AQ4xf0MANlubmA,II,1,3,15,lionn,RU,IRON,16.7,CtXIGE--BubDlnu-IRa8i0-aShS-RFFVoulc509f_bBUyTc,u7EqcmYUd8dqqDYNs_zrlp7Zq8DnHRmdVb1ftkxctDcQaT...,1636,63,1586380433000
1477,v30OeROudYOK9BHdxZQtpt3qCpMur_E4xG7nGnxqja8W_vw,IV,0,32,26,CaduMaverick,BR,PLATINUM,55.2,Mbyy5_dqphEM6vpkLHYYldfPJjORZeLAQy9kOPawLLS5uWk,_mlaxPFj9dvjP2tJXhmvqg64sLGzA6IoONQ7eiCSIIf9fC...,4090,146,1586508988000
1478,OexSfAC7vJmUkFyAFY9_rr74ptYr2IlUfJvrWV5yF04q0MZ-,I,96,14,11,Pr3vent3r,EUW,IRON,56.0,rEX5z8cZTaXwrz0qxRBfH20ZBVmKp_oTU6Q4mi1plTVTj8Q,9nNuhQvGb6GE65xrEFpOsaomU-ls4MOmCBS1kX3B66wo1t...,3008,94,1586552331000


In [25]:
game_list_df = pd.DataFrame()
for index, row in df.head().iterrows():
    sleep(0.1)
    game_list = get_matchlist_by_account_id(row['accountId'],row['server'], only_ranked=True, endIndex=10)
    if game_list is not None:
        game_list_df= game_list_df.append(game_list)

In [26]:
game_list_df=game_list_df.reset_index(drop=True)
game_list_df

,platformId,gameId,champion,queue,season,timestamp,role,lane
0,BR1,1892536945,43,420,13,1585313878993,DUO_SUPPORT,NONE
1,BR1,1890477936,43,420,13,1585171607425,NONE,JUNGLE
2,BR1,1885601127,43,420,13,1584826107594,DUO_SUPPORT,BOTTOM
3,BR1,1884881324,43,420,13,1584768395621,DUO_SUPPORT,TOP
4,BR1,1884305080,43,420,13,1584741765576,DUO_SUPPORT,BOTTOM
5,BR1,1883615046,117,420,13,1584674966347,DUO_SUPPORT,BOTTOM
6,BR1,1883508824,350,420,13,1584672008983,DUO_SUPPORT,BOTTOM
7,BR1,1883530418,117,420,13,1584668961790,DUO_SUPPORT,BOTTOM
8,BR1,1882570517,16,420,13,1584608496104,DUO_SUPPORT,BOTTOM
9,BR1,1882477783,117,420,13,1584595263026,DUO_SUPPORT,BOTTOM


In [27]:
game_list_df = game_list_df.head()
game_list_df

,platformId,gameId,champion,queue,season,timestamp,role,lane
0,BR1,1892536945,43,420,13,1585313878993,DUO_SUPPORT,NONE
1,BR1,1890477936,43,420,13,1585171607425,NONE,JUNGLE
2,BR1,1885601127,43,420,13,1584826107594,DUO_SUPPORT,BOTTOM
3,BR1,1884881324,43,420,13,1584768395621,DUO_SUPPORT,TOP
4,BR1,1884305080,43,420,13,1584741765576,DUO_SUPPORT,BOTTOM


In [42]:
game_list_df.to_csv(matches_sample_route, index=False)

In [43]:
ndf = pd.read_csv(matches_sample_route)
ndf

,platformId,gameId,champion,queue,season,timestamp,role,lane
0,BR1,1892536945,43,420,13,1585313878993,DUO_SUPPORT,NONE
1,BR1,1890477936,43,420,13,1585171607425,NONE,JUNGLE
2,BR1,1885601127,43,420,13,1584826107594,DUO_SUPPORT,BOTTOM
3,BR1,1884881324,43,420,13,1584768395621,DUO_SUPPORT,TOP
4,BR1,1884305080,43,420,13,1584741765576,DUO_SUPPORT,BOTTOM


In [44]:
game_list_df['champion']

AttributeError: 'DataFrame' object has no attribute 'nan'